In [1]:
import pandas as pd
from pickle import dump, load
import copy
import geopandas as geo

In [2]:
list_folders = ['NewYork', 'Massachusetts', 'California', 'Florida', 'Washington', 'Texas']

train= pd.read_pickle('to_process/train_second_week.pkl')
print(list(train.keys()))
for i in list_folders:
    print(i)
    if i=='NewYork': frame=train['New York'];
    else: frame=train[i];
    
    frame.to_csv('%s/train_set.csv'%i, index=False)
test= pd.read_pickle('to_process/test_DG_week2.pkl')
for i in list_folders:
    if i=='NewYork': frame=test['New York']
    else: frame=test[i]
    frame.to_csv('%s/test_set.csv'%i, index=False)

['New York', 'Massachusetts', 'California', 'Florida', 'Texas', 'Washington']
NewYork
Massachusetts
California
Florida
Washington
Texas


In [3]:
list_states = pd.read_pickle('list_states_dataframes_2022_11_02-02_53_58(1).pkl')
list_folders = ['NewYork', 'Massachusetts', 'California', 'Florida', 'Washington', 'Texas']

for i, f in zip(list_states, list_folders):
    display(f)
    frame = copy.copy(i[['name_o', 'name_d', 'total_pop_flow']])
    frame.rename(columns={'name_o':'residence', 'name_d':'workplace', 'total_pop_flow':'commuters'}, inplace=True)
    frame.to_csv('%s/flows.csv'%f, index=False)
    
    frame = copy.copy(i)
    columns = frame.columns
    to_keep = [x for x in columns if x[-2:] == '_o']
    frame = frame[to_keep]
    cities = frame.name_o.unique()
    features = frame[frame.name_o==cities[0]].iloc[0]
    
    for city in cities[1:]:
        sn = frame[frame.name_o==city].iloc[0]
        features = pd.concat([features, sn], axis=1)
        
    renaming = {x : x[:-2] for x in to_keep}
    features = features.transpose().rename(columns=renaming)
    frame = copy.copy(i)
    cities2 = [x for x in frame.name_d.unique() if x not in cities]
    
    if len(cities2) >  0:
        
        columns = frame.columns
        to_keep = [x for x in columns if x[-2:] == '_d']
        frame = frame[to_keep]
        features2 = frame[frame.name_d==cities2[0]].iloc[0]
        
        if len(cities2)>1:
            for city in cities2[1:]:
                sn = frame[frame.name_d==city].iloc[0]
                features2 = pd.concat([features2, sn], axis=1)

        renaming = {x : x[:-2] for x in to_keep}
        try:
            features2 = features2.transpose().rename(columns=renaming)
        except:
            print(features2)
            features2 = pd.DataFrame(features2).transpose().rename(columns=renaming)

        features = pd.concat([features, features2])

    features[['name', 'm', 'area']].to_csv('%s/city2pop.csv'%f, index=False)
    
    features.drop(columns=['area', 'm', 'entertainment_point', 'entertainment_poly'], inplace=True)
    features['other_road_line'] = [0.0]*len(features)
    features['residential_line'] = [0.0]*len(features)
    features = features.reindex(columns=['name','commercial_landuse','food_point', 'food_poly', 'health_point',
                                                         'health_poly', 'industrial_landuse', 'main_road_line',
                                                          'natural_landuse', 'other_road_line', 'residential_landuse',
                                                          'residential_line', 'retail_landuse', 'retail_point', 'retail_poly',
                                                          'school_point', 'school_poly', 'transport_point', 'transport_poly'])
    
    features.to_pickle('%s/features.pkl'%f)

'NewYork'

'Massachusetts'

'California'

'Florida'

'Washington'

'Texas'

In [4]:
for f in list_folders:
    
    city2pop = pd.read_csv('%s/city2pop.csv'%f)
    centroids = geo.read_file('%s/tl.shp'%f)
    city2meta = {}
    
    for r, row in city2pop.iterrows():
        
        name = row['name']
        centroid = centroids[centroids.NAME==name].geometry.centroid
        lat, long = centroid.y.values[0], centroid.x.values[0]
        
        city2meta[name] = {'population':row.m, 'area': row.area, 'centroid':[lat, long]}
        
    with open('%s/city2meta.pkl'%f, 'wb') as out:
        dump(city2meta, out)

/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. 

/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. 

/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. 

/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. 

/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. 

...

/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. 

/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. 

/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. 

/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. 

/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = centroids[centroids.NAME==name].geometry.centroid
/tmp/ipykernel_2841958/1587096421.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. 